# OPTaaS Quick Start

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

## Install the package

In [ ]:
install.packages("devtools")
devtools::install_github("MindFoundry/optaas-r-client")
library(optaas.client)

## Connect to OPTaaS using your API Key

In [5]:
client <- OPTaaSClient$new("https://optaas.mindfoundry.ai", "Your OPTaaS API Key")

## Define your parameters

In [6]:
bool_param <- BoolParameter('my_bool')
cat_param <- CategoricalParameter('my_cat', values=list('a', 'b', 'c'), default='c')

int_param <- IntParameter('my_int', minimum=0, maximum=20)
optional_int_param <- IntParameter('my_optional_int', minimum=-10, maximum=10, optional=TRUE)

parameters <- list(
    bool_param,
    cat_param,
    ChoiceParameter('ints_or_floats', choices=list(
        GroupParameter('ints', items=list(int_param, optional_int_param)),
        GroupParameter('floats', items=list(
            FloatParameter('float1', minimum=0, maximum=1),
            FloatParameter('float2', minimum=0.5, maximum=4.5)
        ))
    ))
)

## Define your scoring function

The argument names in your scoring function must match the parameter names you defined above.

In [7]:
scoring_function <- function(my_bool, my_cat, ints_or_floats) {
    score <- if (isTRUE(my_bool)) 5 else -5
    score <- if (my_cat == 'a') score + 1 else score + 3
    if (!is.null(ints_or_floats$ints)) {
        score <- score + do.call(sum, ints_or_floats$ints)
    } else {
        score <- score * do.call(sum, ints_or_floats$floats)
    }
    score
}

## Create your Task

In [9]:
task <- client$create_task(
    title="Dummy task",
    parameters=parameters,
    goal="min",  # optional (default is "max")
    min_known_score=-22, max_known_score=44  # optional
)

## Run your Task

In [10]:
best_result <- task$run(scoring_function=scoring_function, number_of_iterations=20)
print(best_result)

[1] "Iteration: 1   Score: 8"
[1] "Iteration: 2   Score: 8"
[1] "Iteration: 3   Score: -6"
[1] "Iteration: 4   Score: 12"
[1] "Iteration: 5   Score: -14.1569653059746"
[1] "Iteration: 6   Score: 8"
[1] "Iteration: 7   Score: -4.44460902753463"
[1] "Iteration: 8   Score: -3"
[1] "Iteration: 9   Score: 16"
[1] "Iteration: 10   Score: -14.8980253231124"
[1] "Iteration: 11   Score: 21.8485747749646"
[1] "Iteration: 12   Score: -15.3696042580995"
[1] "Iteration: 13   Score: 28.3611758835602"
[1] "Iteration: 14   Score: -18.8378068272197"
[1] "Iteration: 15   Score: -18.7707738645363"
[1] "Iteration: 16   Score: -18.8109038759233"
[1] "Iteration: 17   Score: -18.0487170869439"
[1] "Iteration: 18   Score: -18.0385126466368"
[1] "Iteration: 19   Score: -18.5484304096542"
[1] "Iteration: 20   Score: -18.2283884619893"
$id
[1] 1865

$configuration
$configuration$id
[1] "ed60ddf1-945d-4728-b049-bb066704a4c4"

$configuration$`_links`
$configuration$`_links`$self
$configuration$`_links`$self$href
[